<div class="toc">
 <ul class="toc-item">
     <li><span><a href="#一、引言" data-toc-modified-id="一、引言">一、引言</a></span></li>
     <li>
         <span><a href="#二、工具调用代码示例" data-toc-modified-id="二、工具调用代码示例">二、工具调用代码示例</a></span>
         <ul class="toc-item">
             <li><span><a href="##2.1 从 Python 函数定义工具接口" data-toc-modified-id="2.1 从 Python 函数定义工具接口">2.1 从 Python 函数定义工具接口</a></span></li>
             <li><span><a href="##2.2 定义自动检索工具" data-toc-modified-id="2.2 定义自动检索工具">2.2 定义自动检索工具</a></span></li>
             <li><span><a href="##2.3 综合的工具选择系统" data-toc-modified-id="2.3 综合的工具选择系统">2.3 综合的工具选择系统</a></span></li>
             </ul>
         </li>
     <li><span><a href="#三、本章小结" data-toc-modified-id="三、本章小结">三、本章小结</a></span></li>
     </ul>
</div>

## 一、引言
***

LLMs 的优点之一是能够与外部环境交互并采取行动。为了实现这一目标，LLM 需要使用有效且可靠的接口，我们称之为**工具调用**。

在基础的 RAG 管道中，LLMs 仅用于生成结果。2. 路由查询引擎 Router Query Engine 教程已经向您展示了如何以略微更复杂的方式使用 LLMs（router），即利用它来选择最佳的查询管道以回答用户的查询。这是工具调用的简化形式。在本章教程中，我们将向您展示如何高效使用 LLM，**不仅正确选择要执行的函数，还要推断要传递给函数的参数**。这使得 LLM 能够理解如何使用向量数据库，而不仅仅是获取其输出。

结果是用户可以提出更多问题，并**通过工具调用获得比标准 RAG 技术更精确的结果**。那么，让我们开始编码吧~


## 二、工具调用代码示例
***

与之前类似，我们首先要设置 `OpenAI API KEY`。我们还将设置并导入 `Nest AsyncIO` 模块。


In [1]:
from helper import get_openai_api_key

OPENAI_API_KEY = get_openai_api_key()

下面两行代码通过**启用对嵌套异步事件循环的支持**，极大地提高了在特定场景下（如 Jupyter Notebook）处理异步操作的可行性和便捷性。

**解决的问题**：通常，在 Python 中，`asyncio` 模块不允许在已经运行的事件循环中再启动另一个事件循环。这是因为 `asyncio` 设计上假设每个线程只会有一个正在运行的事件循环。然而，在某些应用场景，特别是像 Jupyter Notebook 这样的交互式环境中，已经有一个运行中的事件循环了（由于 notebook 自身的需求）。如果用户尝试创建并运行另一个事件循环（例如，使用 `asyncio.run()` ），将会抛出异常。这给基于异步编程模型开发交互式应用带来了不便。因此，`nest_asyncio` 库正是为了解决这一限制而诞生。

In [2]:
import nest_asyncio

nest_asyncio.apply()

### 2.1 从 Python 函数定义工具接口

接下来，我们将进行工具调用的基本回顾和介绍。我们将向您展示**如何从 Python 函数定义工具接口**，LLM 将使用 LlamaIndex 抽象从 **Python 函数的签名**自动推断参数。

为了说明这一点，让我们首先定义两个简单的计算器函数来向您展示工具调用是如何工作的。我们将定义一个 add 函数和一个 mystery 函数。

LlamaIndex 中的核心抽象是函数工具（**FunctionTool**）。**此函数工具会包装您提供给它的任何具体的 Python 函数**，所以，我们看到函数工具接受了这里定义的 add 函数以及一个 mystery 函数（也就是 x + y 乘以 x + y）。您可以看到 add 和 mystery 都有 x 和 y 变量的**类型注释**，还有文档字符串（**docstring**）。这不仅仅是为了编程规范目的，这实际上很重要，因为这些东西将被用作 LLM 的**提示（Prompts）**。目前，LlamaIndex 的函数工具已经与许多 LLM 提供商（包括 OpenAI）的**函数调用功能**进行了原生集成。


In [3]:
from llama_index.core.tools import FunctionTool

def add(x: int, y: int) -> int:
    """Adds two integers together."""
    return x + y

def mystery(x: int, y: int) -> int: 
    """Mystery function that operates on top of two numbers."""
    return (x + y) * (x + y)


add_tool = FunctionTool.from_defaults(fn=add)
mystery_tool = FunctionTool.from_defaults(fn=mystery)

要将工具传递给 LLM，您必须导入 LLM 模块，然后调用 **predict_and_call**。下面的代码片段显式导入了 OpenAI 模块，您会看到模型是 **gpt-3.5-turbo**，然后我们在 LLM 之上调用 predict_and_call 函数。predict_and_call 函数的作用是接受一组工具以及**输入提示字符串或一系列聊天消息**，然后它既能够决定要调用的工具，也能够调用工具本身，并返回最终的响应。

In [4]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo")
response = llm.predict_and_call(
    [add_tool, mystery_tool], 
    "Tell me the output of the mystery function on 2 and 9", 
    verbose=True
)
print("=" * 24)
print(str(response), type(response))

=== Calling Function ===
Calling function: mystery with args: {"x": 2, "y": 9}
=== Function Output ===
121
121 <class 'llama_index.core.chat_engine.types.AgentChatResponse'>


In [5]:
# 中文
from llama_index.core.tools import FunctionTool
from llama_index.llms.openai import OpenAI

def add(x: int, y: int) -> int:
    """将两个整数相加。"""
    return x + y

def mystery(x: int, y: int) -> int: 
    """在两个数字之上运行的神秘函数。"""
    return (x + y) * (x + y)


add_tool = FunctionTool.from_defaults(fn=add)
mystery_tool = FunctionTool.from_defaults(fn=mystery)

llm = OpenAI(model="gpt-3.5-turbo")
response = llm.predict_and_call(
    [add_tool, mystery_tool], 
    "告诉我 mystery 函数对 2 与 9 输出的结果",
    verbose=True
)
print("=" * 24)
print(str(response), type(response))

=== Calling Function ===
Calling function: mystery with args: {"x": 2, "y": 9}
=== Function Output ===
121
121 <class 'llama_index.core.chat_engine.types.AgentChatResponse'>


这里我们看到了调用函数 mystery 的中间步骤。函数传入参数：x 等于 2，y 等于 9。所以我们看到，代码实现调用了正确的工具，也推断出了正确的参数，输出是 121，11 乘以 11 是 121，所以我们得到了正确的答案。但请注意，这个简单的示例实际上是路由（router）的扩展版本。**LLM 不仅选择合适的工具，还要决定给工具给予什么参数**。

下面的 2.2 定义自动检索工具小节，让我们使用这个关键概念在向量搜索之上定义一个稍微复杂一点的智能体层。**LLM 不仅可以选择向量搜索，我们还可以让它推断元数据过滤器**，这是一个结构化的标签列表，有助于返回一组更精确的搜索结果。我们将使用与前面相同的论文 MetaGPT，这次让我们关注节点本身，或者块，因为我们将查看附加到这些块的实际元数据。


### 2.2 定义自动检索工具

加载数据。下载运行代码所需的 **MetaGPT 论文** PDF：

- 直接访问论文链接并下载：https://openreview.net/forum?id=VtmBAGCN7o ，然后将 PDF 文件放置在代码目录下。

- 或者在 jupyter notebook 中直接运行：!wget "https://openreview.net/pdf?id=VtmBAGCN7o" -O metagpt.pdf

下面这行代码的功能是从给定的 URL 下载一个 PDF 文档，并将其以 "metagpt.pdf" 为名保存到当前工作目录下。

- `!`：这个符号通常在 Jupyter Notebook 环境中用来执行系统命令。

- `wget`：这是实际执行下载操作的命令行工具。

- `"https://openreview.net/pdf?id=VtmBAGCN7o"`：这是你想要下载的论文 PDF 的 URL。

- `-O metagpt.pdf`：这个选项指定了下载文件的本地保存名称。`-O`（大写字母 O）后面跟着的 `metagpt.pdf` 是你希望保存下载文件时使用的名称。如果不使用 `-O` 选项，`wget` 会使用 URL 中最后一部分作为文件名来保存下载内容。

In [6]:
!wget "https://openreview.net/pdf?id=VtmBAGCN7o" -O metagpt.pdf

--2024-06-01 21:55:24--  https://openreview.net/pdf?id=VtmBAGCN7o
Resolving openreview.net (openreview.net)... 35.184.86.251
Connecting to openreview.net (openreview.net)|35.184.86.251|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16911937 (16M) [application/pdf]
Saving to: ‘metagpt.pdf’

metagpt.pdf         100%[===================>]  16.13M  34.9KB/s    in 6m 52s  

2024-06-01 22:02:17 (40.1 KB/s) - ‘metagpt.pdf’ saved [16911937/16911937]



In [7]:
from llama_index.core import SimpleDirectoryReader

# 加载文件。使用 LlamaIndex 中的简单目录阅读器来加载此 pdf 文件的解析表示。
documents = SimpleDirectoryReader(input_files=["metagpt.pdf"]).load_data()

这里的每个节点都代表一个块，让我们来看看一个示例块的内容。我们来看看第一个块的内容，通过执行 node.getContent 来完成。

In [8]:
from llama_index.core.node_parser import SentenceSplitter

# 使用句子分割器将这些文档分割成一组偶数块，块大小为 1024。
splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

我们将元数据模式设置为 `all`，这是一个特殊的设置，实际上不仅可以打印节点本身的内容，还可以打印附加到文档的元数据，这些元数据将传播到每个节点。结果打印如下所示：

In [9]:
print(nodes[0].get_content(metadata_mode="all"))

page_label: 1
file_name: metagpt.pdf
file_path: metagpt.pdf
file_type: application/pdf
file_size: 16911937
creation_date: 2024-06-01
last_modified_date: 2024-06-01

Preprint
METAGPT: M ETA PROGRAMMING FOR A
MULTI -AGENT COLLABORATIVE FRAMEWORK
Sirui Hong1∗, Mingchen Zhuge2∗, Jonathan Chen1, Xiawu Zheng3, Yuheng Cheng4,
Ceyao Zhang4,Jinlin Wang1,Zili Wang ,Steven Ka Shing Yau5,Zijuan Lin4,
Liyang Zhou6,Chenyu Ran1,Lingfeng Xiao1,7,Chenglin Wu1†,J¨urgen Schmidhuber2,8
1DeepWisdom,2AI Initiative, King Abdullah University of Science and Technology,
3Xiamen University,4The Chinese University of Hong Kong, Shenzhen,
5Nanjing University,6University of Pennsylvania,
7University of California, Berkeley,8The Swiss AI Lab IDSIA/USI/SUPSI
ABSTRACT
Remarkable progress has been made on automated problem solving through so-
cieties of agents based on large language models (LLMs). Existing LLM-based
multi-agent systems can already solve simple dialogue tasks. Solutions to more
complex tasks, however, 

在这里我们看到，一旦我们打印出来，我们不仅得到了论文首页的解析表示，我们还可以看到**附加在顶部的元数据**。所以这包括几件事。取值为 1 的页标签；**文件名（metagpt.pdf）、文件路径、文件类型、文件大小以及创建和结束日期**。

此外，我们要特别注意页面标签。因为，例如，如果我们实际进入并尝试不同的节点，我们会看到我们实际上得到了不同的页码。所以我们实际上给每个块都添加了页码注释。接下来，**我们将在这些节点上定义一个向量存储索引**。这将基本上在这些节点上构建 RAG 索引管道。它会为每个节点添加一个嵌入，并返回一个查询引擎。

In [10]:
from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex(nodes)
query_engine = vector_index.as_query_engine(similarity_top_k=2)

我们实际上可以尝试通过元数据过滤器查询此 RAG 管道，只是为了向您展示元数据过滤的工作原理。因此，我们导入这个名为 `MetadataFilters` 的对象，然后我们只需指定一个过滤器，其中页面标签的值设置 "2"；另外，similarity_top_k 也等于 2。

因此，我们将其定义为查询引擎，并且我们查询 MetaGPT 论文中的一些高水平结果。如果我们观察一下源节点，一旦我们运行它，我们查询到 MetaGPT 的一些高水平结果。当我们收到响应时，我们首先看一下响应字符串，它概述了 MetaGPT 的总体结果。但至关重要的是，我们将查看页码。我们迭代源节点时，实际上可以打印出**附加到这些源节点的元数据**。我们看到这里的页面标签等于 "2"。因此我们观察到它能够正确过滤页码，**仅将搜索限制为页面标签等于 2 的页面**。这说明我们的查询引擎很聪明，它知道我们只想看标签为 "2" 的论文页面的信息。这样，我们就不会被其他不相关的页面干扰，可以更快地找到我们想要的答案。


In [11]:
from llama_index.core.vector_stores import MetadataFilters

query_engine = vector_index.as_query_engine(
    similarity_top_k=2,
    filters=MetadataFilters.from_dicts(
        [
            {"key": "page_label", "value": "2"}
        ]
    )
)

response = query_engine.query(
    "What are some high-level results of MetaGPT?", 
)

In [12]:
print(str(response))

Some high-level results of MetaGPT include achieving a new state-of-the-art in code generation benchmarks with 85.9% and 87.7% in Pass@1, outperforming other popular frameworks like AutoGPT, LangChain, AgentVerse, and ChatDev. Additionally, MetaGPT excels in handling higher levels of software complexity and providing extensive functionality, as evidenced by achieving a 100% task completion rate in experimental evaluations.


In [13]:
for n in response.source_nodes:
    print(n.metadata)

{'page_label': '2', 'file_name': 'metagpt.pdf', 'file_path': 'metagpt.pdf', 'file_type': 'application/pdf', 'file_size': 16911937, 'creation_date': '2024-06-01', 'last_modified_date': '2024-06-01'}


In [14]:
# 中文
query_engine = vector_index.as_query_engine(
    similarity_top_k=2,
    filters=MetadataFilters.from_dicts(
        [
            {"key": "page_label", "value": "2"}
        ]
    )
)

response = query_engine.query(
    "MetaGPT 取得了哪些高水平的结果？", 
)

In [15]:
print(str(response))
print("-" * 88)
for n in response.source_nodes:
    print(n.metadata)

MetaGPT在代码生成基准测试中取得了85.9%和87.7%的一级通过率，超越了其他流行的框架，如AutoGPT，LangChain，AgentVerse和ChatDev。此外，在处理更高级别的软件复杂性和提供广泛功能方面，MetaGPT也表现出色。
----------------------------------------------------------------------------------------
{'page_label': '2', 'file_name': 'metagpt.pdf', 'file_path': 'metagpt.pdf', 'file_type': 'application/pdf', 'file_size': 16911937, 'creation_date': '2024-06-01', 'last_modified_date': '2024-06-01'}


本教程的最后一节将这个整体检索工具包装成一个函数。这个函数接受查询字符串和页码作为过滤器。**LLM 实际上可以推断要过滤用户查询的页码**，而不是让用户手动指定元数据过滤器。

这里需要注意的是，元数据其实不仅限于页码。**您可以通过 LlamaIndex 抽象定义任何想要的元数据**，如节 IDs、页眉、页脚或其他任何内容。使用许多元数据过滤器的能力在 GPT-4 等更好的大语言模型中尤为突出，因此我们强烈建议读者自行实践一下。

这里我们将定义一个封装它的 Python 函数。我们定义一个名为 **vector_query** 的函数，它接受查询和页码。这允许您对索引执行向量搜索，并指定页码作为元数据过滤器。最后，我们定义了一个向量查询 FunctionTool.from_defaults。因此，我们将 vector_query 函数传递到向量查询工具中，这使得我们可以将其与语言模型一起使用。

因此，让我们借助 gpt-3.5-turbo 调用此工具。我们会发现 LLM 能够推断字符串以及元数据过滤器。我们在向量查询工具上进行了 predict_and_call，并提示了同样的问题：如第二页所述的 MetaGPT 的高水平结果。在这里我们看到 LLM 能够制定正确的查询（MetaGPT 的高水平结果），以及指定页码（取值为 "2"）。结果是我们得到了正确的答案。

In [16]:
from typing import List, Tuple
from llama_index.core.vector_stores import FilterCondition

def vector_query(query: str, page_number: int) -> str:
    """Perform a vector search over an index.

    query (str): the string query to be embedded.
    page_numbers (int): Filter by set of pages. Leave BLANK if we want to perform a vector search over all pages. Otherwise, filter by the set of specified pages.

    """
    print(page_number, type(page_number))
    metadata_dicts = [{"key": "page_label", "value": str(page_number)}]
    print(metadata_dicts)

    query_engine = vector_index.as_query_engine(
        similarity_top_k=2,
        filters=MetadataFilters.from_dicts(
            metadata_dicts,
            condition=FilterCondition.OR
        )
    )
    response = query_engine.query(query)

    return response


vector_query_tool = FunctionTool.from_defaults(
    name="vector_tool",
    fn=vector_query
)

In [17]:
llm = OpenAI(model="gpt-3.5-turbo", temperature=0)
response = llm.predict_and_call(
    [vector_query_tool], 
    "What are the high-level results of MetaGPT as described on page 2?",
    verbose=True
)

=== Calling Function ===
Calling function: vector_tool with args: {"page_number": 2, "query": "high-level results of MetaGPT"}
2 <class 'int'>
[{'key': 'page_label', 'value': '2'}]
=== Function Output ===
MetaGPT achieves a new state-of-the-art (SoTA) in code generation benchmarks with 85.9% and 87.7% in Pass@1. It stands out in handling higher levels of software complexity and offering extensive functionality. In experimental evaluations, MetaGPT achieves a 100% task completion rate, demonstrating robustness and efficiency in terms of time and token costs.


In [19]:
# 中文
def vector_query(query: str, page_number: int) -> str:
    """对索引进行向量搜索。

    query (str)：要进行嵌入的字符串查询。
    page_numbers (int)：按页面集过滤。如果要在所有页面上进行向量搜索，则留空。否则，按指定的页面集过滤。

    """
    print(page_number, type(page_number))
    metadata_dicts = [{"key": "page_label", "value": str(page_number)}]
    print(metadata_dicts)

    query_engine = vector_index.as_query_engine(
        similarity_top_k=2,
        filters=MetadataFilters.from_dicts(
            metadata_dicts,
            condition=FilterCondition.OR
        )
    )
    response = query_engine.query(query)

    return response


vector_query_tool = FunctionTool.from_defaults(
    name="vector_tool",
    fn=vector_query
)

llm = OpenAI(model="gpt-3.5-turbo", temperature=0)
response = llm.predict_and_call(
    [vector_query_tool], 
    "第 2 页中描述的 MetaGPT 的高水平结果是什么？请用中文回答。",
    verbose=True
)

=== Calling Function ===
Calling function: vector_tool with args: {"page_number": 2, "query": "MetaGPT \u7684\u9ad8\u6c34\u5e73\u7ed3\u679c"}
2 <class 'int'>
[{'key': 'page_label', 'value': '2'}]
=== Function Output ===
MetaGPT 在代码生成基准测试中取得了新的技术水平，Pass@1 分别为 85.9% 和 87.7%，在处理更高级别的软件复杂性和提供广泛功能方面表现出色。在实验评估中，MetaGPT 实现了 100% 的任务完成率，展示了设计的稳健性和效率。


我们可以快速验证源节点，可以看到源节点的页面标签，有一个返回的源节点，它的页面标签是 "2"。

In [20]:
for n in response.source_nodes:
    print(n.metadata)

{'page_label': '2', 'file_name': 'metagpt.pdf', 'file_path': 'metagpt.pdf', 'file_type': 'application/pdf', 'file_size': 16911937, 'creation_date': '2024-06-01', 'last_modified_date': '2024-06-01'}


### 2.3 综合的工具选择系统

最后，我们可以从 2. 路由查询引擎 Router Query Engine 教程中的 router 示例中引入摘要工具，并将其与向量检索工具有机结合起来，创建这个综合的工具选择系统。所以，下面这段代码只是在相同的节点集上设置了一个摘要索引，并用一个类似于 2. 路由查询引擎 Router Query Engine 教程中的摘要工具将其包装起来。


In [21]:
from llama_index.core import SummaryIndex
from llama_index.core.tools import QueryEngineTool

summary_index = SummaryIndex(nodes)
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
summary_tool = QueryEngineTool.from_defaults(
    name="summary_tool",
    query_engine=summary_query_engine,
    description=(
        "Useful if you want to get a summary of MetaGPT."
    ),
)

现在我们再次尝试工具调用。这样一来，**LLM 要做的任务略有提高难度**，它不仅要实际选择正确的工具，还要推断函数参数。我们询问论文第 8 页上 MetaGPT 与 ChatDev 的比较情况。我们看到它实际上还是调用了一个向量工具，页码等于 "8"，而且它能够给出正确的答案。


In [22]:
response = llm.predict_and_call(
    [vector_query_tool, summary_tool], 
    "What are the MetaGPT comparisons with ChatDev described on page 8?", 
    verbose=True
)

=== Calling Function ===
Calling function: vector_tool with args: {"page_number": 8, "query": "MetaGPT comparisons with ChatDev"}
8 <class 'int'>
[{'key': 'page_label', 'value': '8'}]
=== Function Output ===
MetaGPT outperforms ChatDev on the SoftwareDev dataset in various metrics. For example, MetaGPT achieves a higher score in executability, takes less time for software generation, uses more tokens but requires fewer tokens to generate one line of code compared to ChatDev. Additionally, MetaGPT demonstrates better performance in code statistics and human revision cost when compared to ChatDev.


我们还可以通过打印出来源节点来验证这一点。

In [23]:
for n in response.source_nodes:
    print(n.metadata)

{'page_label': '8', 'file_name': 'metagpt.pdf', 'file_path': 'metagpt.pdf', 'file_type': 'application/pdf', 'file_size': 16911937, 'creation_date': '2024-06-01', 'last_modified_date': '2024-06-01'}


In [27]:
# 中文
summary_index = SummaryIndex(nodes)
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
summary_tool = QueryEngineTool.from_defaults(
    name="summary_tool",
    query_engine=summary_query_engine,
    description=(
        "如果您想了解 MetaGPT 的概要，这个工具会很有用。"
    ),
)

response = llm.predict_and_call(
    [vector_query_tool, summary_tool], 
    "第 8 页描述的 MetaGPT 与 ChatDev 的对比分析结果是什么？必须用中文回答。", 
    verbose=True
)

for n in response.source_nodes:
    print(n.metadata)

=== Calling Function ===
Calling function: vector_tool with args: {"page_number": 8, "query": "MetaGPT \u4e0e ChatDev \u7684\u5bf9\u6bd4\u5206\u6790\u7ed3\u679c"}
8 <class 'int'>
[{'key': 'page_label', 'value': '8'}]
=== Function Output ===
MetaGPT 在软件开发数据集中几乎在所有指标上均优于 ChatDev。例如，在可执行性方面，MetaGPT 获得了3.75的分数，接近4（完美）。此外，MetaGPT 生成一行代码所需的时间更短（503秒），明显少于 ChatDev。考虑到代码统计和人工修订成本，MetaGPT 也明显优于 ChatDev。虽然 MetaGPT 需要更多的标记（24,613或31,255比19,292多），但只需要126.5/124.3个标记来生成一行代码。相比之下，ChatDev 使用了248.9个标记。这些结果突显了在多个代理之间的协作中 SOP 的好处。
{'page_label': '8', 'file_name': 'metagpt.pdf', 'file_path': 'metagpt.pdf', 'file_type': 'application/pdf', 'file_size': 16911937, 'creation_date': '2024-06-01', 'last_modified_date': '2024-06-01'}


最后，我们可以问一个问题，即**这篇论文的摘要是什么**，以显示 LLM 在必要时仍然能够选择摘要工具。我们看到它给出了正确的回应。

In [36]:
summary_index = SummaryIndex(nodes)
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
summary_tool = QueryEngineTool.from_defaults(
    name="summary_tool",
    query_engine=summary_query_engine,
    description=(
        "Useful if you want to get a summary of MetaGPT."
    ),
)

response = llm.predict_and_call(
    [vector_query_tool, summary_tool], 
    "What is a summary of the MetaGPT paper?",
    verbose=True
)

=== Calling Function ===
Calling function: summary_tool with args: {"input": "Summary of the MetaGPT paper"}
=== Function Output ===
The MetaGPT paper introduces a meta-programming framework that utilizes Standardized Operating Procedures (SOPs) to enhance multi-agent systems based on Large Language Models (LLMs). It focuses on role specialization, structured communication interfaces, and executable feedback mechanisms to improve code generation quality during runtime. The framework involves agents like Product Managers, Architects, Engineers, and QA Engineers, each contributing specialized outputs to efficiently complete software development tasks. Additionally, it introduces the concept of AgentStore for creating and developing agents within the framework. The paper discusses the transformation of abstract requirements into detailed software designs, emphasizing the importance of structured messaging and feedback mechanisms. It addresses challenges such as reducing code hallucination

In [45]:
# 中文
summary_index = SummaryIndex(nodes)
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
summary_tool = QueryEngineTool.from_defaults(
    name="summary_tool",
    query_engine=summary_query_engine,
    description=(
        "如果您想了解 MetaGPT 的概要，这个工具会很有用。"
    ),
)

# 发现使用 LlamaIndex 框架，即使是中文 Prompt，仍可能出现不遵循指令的情况。多次调试，输出结果仍是英文。
# 因此，感兴趣的读者可以换一篇中文的论文 PDF 来做实验！
response = llm.predict_and_call(
    [vector_query_tool, summary_tool], 
    "用中文输出 MetaGPT 的摘要。",
    verbose=True
)

=== Calling Function ===
Calling function: summary_tool with args: {"input": "MetaGPT"}
=== Function Output ===
MetaGPT is a meta-programming framework that enhances problem-solving capabilities in multi-agent systems based on Large Language Models (LLMs). It incorporates role specialization, structured communication interfaces, and efficient sharing mechanisms to facilitate collaboration among agents. MetaGPT also utilizes an executable feedback mechanism to improve code generation quality during runtime. In experiments, MetaGPT has shown state-of-the-art performance on various benchmarks, outperforming other frameworks in tasks like code generation and software development. Additionally, MetaGPT is a framework that utilizes natural language programming to generate software applications based on provided prompts and requirements. It simplifies the process of transforming abstract requirements into detailed class and function designs through a specialized division of labor and standard

## 三、本章小结
***

🧍🐒🤖 人与大多数动物的根本区别在于人能制造和使用工具。同样地，智能体也能利用各种外部工具来实现功能扩展和效率提升。

在本教程中，我们展示了如何使用 LLM 来选择要执行的函数，并推断要传递给函数的参数。通过工具调用，LLM 不仅能够选择合适的工具，还能决定要给工具什么参数。这使得用户能够提出更多问题，并通过工具调用获得比标准 RAG 技术更精确的结果。

教程中还展示了如何在向量搜索之上定义一个稍微复杂一点的智能体层。LLM 可以选择向量搜索，并推断元数据过滤器，以返回更精确的搜索结果。最后，我们展示了如何将整体检索工具封装成一个函数，使 LLM 能够推断用户查询的元数据过滤器。通过 LLM 索引抽象，可以定义各种元数据，如章节 IDs、页码、页眉、页脚等。

✨️🎉🚀 那么，3. 工具调用 Tool Calling 就到这里啦~ ✨️🎉🚀 

🤖 下一章 4. Building an Agent Reasoning Loop，我们将向您展示如何构建智能体推理闭环！🤖